In [ ]:
!pip install ipython-autotime
%load_ext autotime

Nhược điểm chung của Random Search và Grid Search là đều hoạt động theo một cách tìm kiếm mà không biết trước thông tin cơ bản nào (uninformed). Giả sử một số lần test phát hiện ra một số hyperparameters nhất định không ảnh hưởng đến kết quả hoặc các khoảng giá trị nhất định không thực sự hiệu quả thì những thông tin này không được lan truyền tới các lần tìm kiếm sau. Vì lý do này, Halving Search ra đời với HalvingGridSearch và HalvingRandomSearch được sử dụng để tìm kiếm trên một parameter space bằng cách áp dụng cơ chế thực nghiệm với những lần chia đôi liên tiếp trên Grid Search và Random Search.

Sau khi giảm không gian xuống một nửa, một số lượng lớn các kết hợp hyperparameters được đánh giá trong một vòng thực nghiệm ban đầu nhưng sử dụng tài nguyên tính toán ít hơn. Điều này đạt được bằng cách chạy thử nghiệm trên một mẫu phụ của một vài trường hợp từ dataset. Vòng đầu tiên cho phép lựa chọn một tập hợp con của các hyperparameter, với những hyperparameter tốt nhất trên vòng 1 sẽ được sử dụng cho vòng thứ 2 với kích thước dataset được tăng lên. Và các vòng tiếp theo làm tương tự.

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

time: 4.74 s (started: 2023-05-28 13:43:48 +00:00)


In [3]:
X, y = make_classification(n_samples=300, n_features=50,
                          n_informative=10,
                          n_redundant=25, n_repeated=15,
                          n_clusters_per_class=5,
                          flip_y=0.05, class_sep=0.5,
                          random_state=0)

time: 25.5 ms (started: 2023-05-28 13:43:53 +00:00)


In [5]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn import svm
from sklearn.utils.fixes import loguniform
from sklearn import model_selection

svc = svm.SVC()
svc = svm.SVC(probability=True, random_state=1)

search_dict = {'kernel': ['linear', 'rbf'],
                'C': loguniform(1, 1000),
                'gamma': loguniform(0.0001, 0.1)
              }

scorer = 'accuracy'

time: 22.3 ms (started: 2023-05-28 13:44:51 +00:00)


In [6]:
search_func = HalvingRandomSearchCV(estimator=svc,
                                  param_distributions=search_dict,
                                  resource='n_samples',
                                  max_resources=100,
                                  aggressive_elimination=True,
                                  scoring=scorer,
                                  n_jobs=-1,
                                  cv=5,
                                  random_state=0)

time: 764 µs (started: 2023-05-28 13:44:56 +00:00)


In [7]:
search_func.fit(X, y)
print (search_func.best_params_)
print (search_func.best_score_)

{'C': 86.63279761354555, 'gamma': 0.002054762512591133, 'kernel': 'linear'}
0.6166666666666667
time: 1min 43s (started: 2023-05-28 13:45:07 +00:00)
